# Time Series Modelling
I can manually put the lagged variables in the features and then mask the ones for the k values of the model.
Will this work? Parameters are trained for all the values but we might need seperate parameters for each.

- It will not work. To train ARIMA, I need different parameter trineers for each model which is incredibly expensive computationally. We an try a linear approach which might be faster
- Global XGBoost might work
    - Boosting is an ensemble technique where new models are added to correct the errors made by existing models. Models are added sequentially until no further improvements can be made. 
    - Gradient boosting is an approach where new models are created that predict the residuals or errors of prior models and then added together to make the final prediction. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.

In [3]:
import pandas as pd
from pathlib import Path


In [4]:
data = pd.read_parquet(Path("data") / "train.parquet")
external_data = pd.read_csv("external_data.csv")

In [ ]:
def merge_external_data(main_data, external_data):
    data = main_data.join(external_data.set_index("date"), on="date") 
    
    
    return data    